In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import pymc.sampling_jax
import bambi as bmb
import scipy.stats as stats
from scipy.stats import gaussian_kde
from sklearn.preprocessing import scale
import arviz as az
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

/Users/corrado/mambaforge/envs/pymc_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [8]:
# Initialize a random number generator for reproducibility
rng = np.random.default_rng(seed=42)

# Generate date range
dates = pd.date_range(start="2020-05-01", end="2020-07-01")
n_dates = len(dates)

# Cities and their average temperatures
cities = ["Berlin", "Paris", "Rome"]
avg_temps = [15, 17, 20]
n_cities = len(cities)

# Pre-allocate a NumPy array to hold the temperature data
temperature_array = np.zeros(n_dates * n_cities)

# Populate the temperature array
for i, avg_temp in enumerate(avg_temps):
    temperature_array[i * n_dates : (i + 1) * n_dates] = rng.normal(
        loc=avg_temp, scale=2, size=n_dates
    )

# Create date and city columns, repeating as necessary
date_column = np.tile(dates, n_cities)
city_column = np.repeat(cities, n_dates)

# Create a DataFrame in long format
df_data_long = pd.DataFrame(
    {"date": date_column, "city": city_column, "temperature": temperature_array}
)

# Show first few rows
print(df_data_long.head())

        date    city  temperature
0 2020-05-01  Berlin    15.609434
1 2020-05-02  Berlin    12.920032
2 2020-05-03  Berlin    16.500902
3 2020-05-04  Berlin    16.881129
4 2020-05-05  Berlin    11.097930


In [9]:
# The "coordinates" are the unique values that these dimensions can take
coords = {
    "date": pd.Categorical(df_data_long["date"]).categories,
    "city": pd.Categorical(df_data_long["city"]).categories,
}
print(coords)

{'date': DatetimeIndex(['2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04',
               '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08',
               '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12',
               '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16',
               '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
               '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24',
               '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28',
               '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01',
               '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05',
               '2020-06-06', '2020-06-07', '2020-06-08', '2020-06-09',
               '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-13',
               '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17',
               '2020-06-18', '2020-06-19', '2020-06-20', '2020-06-21',
               '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25

In [12]:
RANDOM_SEED = 123  

with pm.Model(coords=coords) as model:
    # Constant Data
    data = pm.Data("observed_temp", df_data_long["temperature"], dims=("obs_id"))
    
    # Coordinates for the observed data
    date_idx = pd.Categorical(df_data_long["date"]).codes
    city_idx = pd.Categorical(df_data_long["city"]).codes
    
    # Priors
    europe_mean = pm.Normal("europe_mean_temp", mu=15.0, sigma=3.0)
    city_offset = pm.Normal("city_offset", mu=0.0, sigma=3.0, dims="city")
    
    # Expected city temperature
    city_temperature = pm.Deterministic(
        "expected_city_temp", europe_mean + city_offset[city_idx], dims="obs_id"
    )
    
    # Model Error
    sigma = pm.Exponential("sigma", 1)
    
    # Likelihood
    pm.Normal("temperature", mu=city_temperature, sigma=sigma, observed=data, dims=("obs_id"))
    
    # Sampling
    idata = pm.sampling_jax.sample_numpyro_nuts()


/Users/corrado/mambaforge/envs/pymc_env/lib/python3.11/site-packages/pymc/data.py:433: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(


Compiling...
Compilation time =  0:00:01.439806
Sampling...


Compiling.. :   0%|          | 0/2000 [00:00<?, ?it/s]








Running chain 0:  65%|██████▌   | 1300/2000 [00:03<00:00, 12582.63it/s]

Running chain 3: 100%|██████████| 2000/2000 [00:03<00:00, 599.34it/s]  


Sampling time =  0:00:04.356327
Transforming variables...
Transformation time =  0:00:00.190498


In [13]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
europe_mean_temp,16.751,1.574,13.730,19.573,0.055,0.039,831.0,1104.0,1.0
city_offset[Berlin],-1.684,1.585,-4.583,1.358,0.055,0.039,823.0,1118.0,1.0
city_offset[Paris],-0.026,1.582,-2.873,2.989,0.055,0.039,837.0,1141.0,1.0
city_offset[Rome],3.248,1.583,0.236,6.139,0.055,0.039,836.0,1080.0,1.0
sigma,1.746,0.090,1.576,1.914,0.002,0.002,1385.0,1423.0,1.0
...,...,...,...,...,...,...,...,...,...
expected_city_temp[181],19.999,0.219,19.581,20.405,0.003,0.002,4453.0,3053.0,1.0
expected_city_temp[182],19.999,0.219,19.581,20.405,0.003,0.002,4453.0,3053.0,1.0
expected_city_temp[183],19.999,0.219,19.581,20.405,0.003,0.002,4453.0,3053.0,1.0
expected_city_temp[184],19.999,0.219,19.581,20.405,0.003,0.002,4453.0,3053.0,1.0
